In [1]:
from scipy.stats import truncnorm
import pandas as pd
import numpy as np
import itertools
import datetime
import tqdm
import sys
import os

def flatten_list(list_array):
    return list(itertools.chain(*list_array))

sys.path.insert(0,"../")
from global_config import config

results_dir           = config.get_property('results_dir')
data_dir              = config.get_property('data_dir')
data_db_dir           = config.get_property('data_db_dir')
feb_hosp_records_path = os.path.join(data_db_dir, 'long_files_8_25_2021')
path_to_save          = os.path.join(results_dir, "real_testing", "community")


In [2]:
from utils.data_utils import load_movement_df, ward2size
from utils.plot_utils import *

path_to_data = os.path.join('..', '..', 'data')

# load scenarios for synthetic inferences
scenarios_df              = pd.read_csv(os.path.join(path_to_data, 'scenarios.csv'))
movement_df, ward2cluster = load_movement_df(path_to_data, True) # movement data
ward2size                 = ward2size(movement_df)
ward2size                 = {r.ward_id: r.num_patients for idx_r, r in ward2size.iterrows()}

/var/folders/7p/jp6xqkvn5wb6ddl1fn0bhs980000gn/T/ipykernel_63586/1033347647.py:8: DtypeWarning: Columns (0,1,4,8) have mixed types.Specify dtype option on import or set low_memory=False.
  movement_df, ward2cluster = load_movement_df(path_to_data, True) # movement data


In [3]:
date_min         = pd.to_datetime("2020-02-01")
date_max         = pd.to_datetime("2021-02-28")
dates_simulation = pd.date_range(start=date_min, end=date_max)

γ_prior  = [0.01, 0.9]
β_prior  = [0.001, 0.05]

param_prior_dict      = {}
param_prior_dict["γ"] = γ_prior
param_prior_dict["β"] = β_prior

# Agent based model settings.
abm_settings                     = {}
abm_settings["num_patients"]     = movement_df.mrn_id.unique().shape[0]
abm_settings["num_wards"]        = movement_df.ward_id.unique().shape[0]
abm_settings["num_clusters"]     = len(set(list(ward2cluster.values())))
abm_settings["dates"]            = dates_simulation
abm_settings["num_ensembles"]    = 300

# Iterated filtering settings.
if2_settings                     = {}
if2_settings["num_params"]       = len(param_prior_dict)
if2_settings["num_observations"] = len(set(list(ward2cluster.values())))
if2_settings["lambda_inf"]       = 1.01        # Inflation for the EAKF.
if2_settings["num_iters_mif"]    = 20          # Number of iterations.
if2_settings["alpha_mif"]        = 0.8         # Variance shrinking factor.
if2_settings["type_cooling"]     = "geometric" # Type of cooling.
if2_settings["num_ensembles"]    = 300
if2_settings["oev_variance"]     = 0.2


## Simulate scenarios for synthetic inferences
Main manuscript uses 6% but 4% and 8% are used for sensitivity analyses

In [5]:
from utils.model_utils import simulate_model

for ρ in [6, 4, 8]:
    for idx_row, row in scenarios_df.iterrows():

        θ      = {}
        θ['γ'] = row['γ']
        θ['β'] = row['β']
        θ['ρ'] = ρ / 100

        path_to_scenario = os.path.join('..', '..', 'results', 'synthetic_inferences', f'ρ_{ρ}%', row.name_scenario)
        os.makedirs(path_to_scenario, exist_ok=True)
        name_sims_save   = f"simulation_infer.npz"

        #if os.path.isfile(os.path.join(path_to_scenario, name_sims_save)):
        #    continue

        print("***-***-***-***-***-***")
        print(f"Simulating model for scenario {row.name_scenario}... ρ={ρ}%")
        print("***-***-***-***-***-***")

        ward_colonized, ward_nosocomial, ward_imported, ward_positive, ward_negative,\
        cluster_colonized, cluster_nosocomial, cluster_imported, cluster_positive, cluster_negative \
            = simulate_model(movement_df, ward2size, ward2cluster, θ, abm_settings)

        positive_sim   = ward_positive.sum(-2)
        idx_use        = np.argsort(np.sum(positive_sim, 0))[abm_settings["num_ensembles"]//2]


        np.savez_compressed(os.path.join(path_to_scenario, name_sims_save),
                                        ward_colonized           = ward_colonized,
                                        ward_nosocomial          = ward_nosocomial,
                                        ward_imported            = ward_imported,
                                        ward_positive            = ward_positive,
                                        ward_negative            = ward_negative,
                                        cluster_colonized        = cluster_colonized,
                                        cluster_nosocomial       = cluster_nosocomial,
                                        cluster_imported         = cluster_imported,
                                        cluster_positive         = cluster_positive,
                                        cluster_negative         = cluster_negative,
                                        idx_use                  = idx_use)

        #obs_w_chunk_df, neg_w_chunk_df = create_obs_infer(cluster_positive, cluster_negative, abm_settings, if2_settings)

***-***-***-***-***-***
Simulating model for scenario scenario1... ρ=6%
***-***-***-***-***-***


394it [13:06,  2.00s/it]


***-***-***-***-***-***
Simulating model for scenario scenario2... ρ=6%
***-***-***-***-***-***


394it [16:06,  2.45s/it]


***-***-***-***-***-***
Simulating model for scenario scenario3... ρ=6%
***-***-***-***-***-***


...

394it [12:08,  1.85s/it]


***-***-***-***-***-***
Simulating model for scenario scenario9... ρ=8%
***-***-***-***-***-***


394it [12:32,  1.91s/it]


***-***-***-***-***-***
Simulating model for scenario scenario10... ρ=8%
***-***-***-***-***-***


394it [13:16,  2.02s/it]


In [ ]:
for idx_row, row in scenarios_df.iterrows():
    θ      = {}
    θ['γ'] = row['γ']
    θ['β'] = row['β']
    θ['ρ'] = ρ / 100

    path_to_scenario = os.path.join('..', '..', 'results', 'synthetic_inferences', f'ρ_{ρ}%', row.name_scenario)
    name_sims_save   = f"simulation_infer.npz"
    sim_samples      = np.load(os.path.join(path_to_scenario, name_sims_save))

    cluster_positive = sim_samples['cluster_positive'][:, :, sim_samples['idx_use']]
    cluster_negative = sim_samples['cluster_negative'][:, :, sim_samples['idx_use']]

In [ ]:
def IF2_eakf_ABM(data_infer, pos_obs_df, neg_obs_df, movement_df, param_prior, if2_settings, abm_settings, ward2cluster, ward2size, perturb_time=True, model=model_infer):

    obs_w_chunk_df = pos_obs_df
    neg_w_chunk_df = neg_obs_df
    cooling_factor = cooling(if2_settings["num_iters_mif"], type_cool=if2_settings["type_cooling"], cooling_factor=if2_settings["alpha_mif"])

    param_range    = np.array([v for k, v in param_prior_dict.items()])
    std_param      = param_range[:,1] - param_range[:,0]
    SIG            = std_param ** 2 / 4; #  initial covariance of parameters

    # Perturbation is proportional to the prior range of search.
    perturbation     = np.array([std_param % list(np.round(std_param)+0.1)]).T
    num_steps        = len(obs_w_chunk_df)

    param_mean_iter  = np.full((if2_settings["num_params"],       if2_settings["num_iters_mif"]+1), np.nan)                                         # Array to store posterior parameters in iterations.
    para_post_all    = np.full((if2_settings["num_params"],       if2_settings["num_ensembles"], num_steps, if2_settings["num_iters_mif"]), np.nan) # Array to store posterior parameters.
    param_iter       = np.full((if2_settings["num_params"],       if2_settings["num_ensembles"], if2_settings["num_iters_mif"]), np.nan)

    obs_post_all_pos = np.full((if2_settings["num_observations"], if2_settings["num_ensembles"], num_steps, if2_settings["num_iters_mif"]), np.nan) # Array for store posterior observations
    obs_post_all_neg = np.full((if2_settings["num_observations"], if2_settings["num_ensembles"], num_steps, if2_settings["num_iters_mif"]), np.nan) # Array for store posterior observations
    p_priors_all     = np.full((if2_settings["num_params"],       if2_settings["num_ensembles"], if2_settings["num_iters_mif"]), np.nan)

    dates_assimilation     = obs_w_chunk_df.index.get_level_values(0).values
    dates_assimilation[-1] = abm_settings["dates"][-1]

    α            = np.random.uniform( 1/365, 1/175, size=(abm_settings["num_patients"], if2_settings["num_ensembles"]))
    perturb_time = True

    print(f"Running MIF  \n")
    for n in tqdm(range(if2_settings["num_iters_mif"])):
        if n==0: # Initial IF iteration
            p_prior               = sample_params_uniform(param_prior_dict, num_ensembles=if2_settings["num_ensembles"])
            param_mean_iter[:, n] = np.mean(p_prior, -1)
            p_priors_all[:,:,n]   = p_prior

        else:
            params_mean           = param_mean_iter[:,n]
            params_var            = SIG * cooling_factor[n]
            p_prior               = sample_params_normal(param_prior_dict, params_mean, params_var, num_ensembles=if2_settings["num_ensembles"])
            p_priors_all[:,:,n]   = p_prior

        patients_state    = np.zeros((abm_settings["num_patients"], if2_settings["num_ensembles"]))
        param_post_time   = np.full((if2_settings["num_params"], if2_settings["num_ensembles"], num_steps), np.nan)

        obs_post_time_pos = np.full((abm_settings["num_clusters"], if2_settings["num_ensembles"], num_steps), np.nan)
        obs_post_time_neg = np.full((abm_settings["num_clusters"], if2_settings["num_ensembles"], num_steps), np.nan)

        idx_date_update   = 0

        # Init observation arrays.
        chunk_pos_t = np.zeros((abm_settings["num_clusters"], if2_settings["num_ensembles"]))
        chunk_neg_t = np.zeros((abm_settings["num_clusters"], if2_settings["num_ensembles"]))

        for idx_date, date in enumerate(abm_settings["dates"]):
            # Integrate model
            γ = p_prior[0, :]
            β = p_prior[1, :]

            movement_date = movement_df.loc[date]
            patients_state, _, chunk_pos, _, chunk_neg = model(patients_state, γ, β, α, movement_date, ward2size, ward2cluster)

            chunk_pos_t += chunk_pos
            chunk_neg_t += chunk_neg

            if pd.to_datetime(date) == pd.to_datetime(dates_assimilation[idx_date_update]):
                # Perturb parameters according to the define mapping
                if perturb_time:
                    # Transform parameters for perturbation
                    std_params = perturbation * cooling_factor[n]
                    p_prior    = random_walk_perturbation(p_prior, std_params, if2_settings["num_params"], if2_settings["num_ensembles"])

                # Inflate parameters
                p_prior = inflate_ensembles(p_prior, inflation_value=if2_settings["lambda_inf"], num_ensembles=if2_settings["num_ensembles"])
                p_prior = checkbound_params(param_prior_dict, p_prior, num_ensembles=if2_settings["num_ensembles"])

                # first adjust using only positives
                oev_pos    = obs_w_chunk_df.loc[date][[f"oev_{idx_chunk}" for idx_chunk in range(abm_settings["num_clusters"])]].values
                pos_time   = obs_w_chunk_df.loc[date][[f"pos_{idx_chunk}" for idx_chunk in range(abm_settings["num_clusters"])]].values

                # then adjust using negatives
                oev_neg    = neg_w_chunk_df.loc[date][[f"oev_{idx_chunk}" for idx_chunk in range(abm_settings["num_clusters"])]].values
                neg_time   = neg_w_chunk_df.loc[date][[f"pos_{idx_chunk}" for idx_chunk in range(abm_settings["num_clusters"])]].values

                param_post = p_prior.copy()


                param_post, obs_post_pos = eakf_step_multi_obs(param_post, chunk_pos_t, np.expand_dims(pos_time, -1),  np.expand_dims(oev_pos, -1), param_prior_dict, int(if2_settings["num_observations"] )) # Use both positives to adjust
                param_post               = checkbound_params(param_prior_dict, params_ens=param_post, num_ensembles=if2_settings["num_ensembles"])

                param_post, obs_post_neg = eakf_step_multi_obs(param_post, chunk_neg_t, np.expand_dims(neg_time, -1),  np.expand_dims(oev_neg, -1), param_prior_dict, int(if2_settings["num_observations"] )) # Use negatives to adjust
                param_post               = checkbound_params(param_prior_dict, params_ens=param_post, num_ensembles=if2_settings["num_ensembles"])
                obs_post_time_pos[:, :, idx_date_update]    = obs_post_pos
                obs_post_time_neg[:, :, idx_date_update]    = obs_post_neg

                # Use posterior as next prior
                p_prior = param_post.copy()
                param_post_time[:,:,idx_date_update]  = param_post
                idx_date_update += 1

                chunk_pos_t = np.zeros((abm_settings["num_clusters"], if2_settings["num_ensembles"]))
                chunk_neg_t = np.zeros((abm_settings["num_clusters"], if2_settings["num_ensembles"]))

        para_post_all[:,:,:,n]    = param_post_time
        param_mean_iter[:,n+1]    = param_post_time.mean(-1).mean(-1)
        obs_post_all_pos[:,:,:,n] = obs_post_time_pos
        obs_post_all_neg[:,:,:,n] = obs_post_time_neg

    return obs_w_chunk_df, neg_w_chunk_df, obs_post_all_pos, obs_post_all_neg, para_post_all, param_iter, param_mean_iter